# Climate Data Acquisition: ERA5-Land

**Goal:** To download our first set of climate stressor data from the ERA5-Land dataset via the CDS API.

**Plan:**
1.  Download a test sample: daily **temperature** and **precipitation** data.
2.  Focus on a specific region: the US Midwest.
3.  Cover our full time period: 1981-2016.

In [ ]:
# Cell 1: Download Script for ERA5-Land Data (with Year-Month Loop)
import cdsapi
import os
import time

# --- Configuration ---
# We'll save each month's data into a yearly folder to keep things organized.
output_dir_base = '../data/climate_raw/usa_era5_land_hourly/'
os.makedirs(output_dir_base, exist_ok=True)

# Define the years and months we want to download
years_to_download = range(1981, 2017)
months_to_download = range(1, 13)

# --- API Request Nested Loop ---
c = cdsapi.Client()

print(f"Starting download for {len(years_to_download)} years and {len(months_to_download)} months per year.")
print("This will make a total of {} requests.".format(len(years_to_download) * len(months_to_download)))

for year in years_to_download:
    for month in months_to_download:
        # Create a yearly sub-folder
        year_dir = os.path.join(output_dir_base, str(year))
        os.makedirs(year_dir, exist_ok=True)
        
        # Define a unique output file for each year and month
        output_file = os.path.join(year_dir, f'era5_land_usa_{year}_{month:02d}.nc')
        
        # Check if the file already exists to avoid re-downloading
        if os.path.exists(output_file):
            print(f"File for {year}-{month:02d} already exists. Skipping.")
            continue

        print(f"\nSubmitting request for: {year}-{month:02d}")
        
        try:
            request_dictionary = {
                'format': 'netcdf',
                'variable': [
                    '2m_temperature', 'total_precipitation',
                ],
                'year': str(year),
                'month': f'{month:02d}', # Request only the current month
                'day': [f'{d:02d}' for d in range(1, 32)],
                'time': [f'{h:02d}:00' for h in range(0, 24)],
                'area': [50, -105, 25, -80], # North, West, South, East
            }

            c.retrieve(
                'reanalysis-era5-land',
                request_dictionary,
                output_file
            )
            print(f"Download for {year}-{month:02d} complete!")
            
            # A small polite pause between requests
            time.sleep(1)

        except Exception as e:
            print(f"\nAn error occurred for {year}-{month:02d}: {e}")
            print("The script will continue with the next month.")
            continue

print("\n--- All downloads finished! ---")